# Setup

In [ ]:
import jnius_config

In [ ]:
jnius_config.set_classpath("../../java/target/demo-python-java-api-1.0-SNAPSHOT-jar-with-dependencies.jar")

In [ ]:
import jnius

import cProfile
import time
import pandas as pd
import random

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import sys 
sys.path.append('..')
import api.demojnius

In [ ]:
Simulation = jnius.autoclass('ch.dubernet.demopythonapi.simulation.Simulation')
BenchmarkEventHandler = jnius.autoclass('ch.dubernet.demopythonapi.simulation.events.BenchmarkEventHandler')

JumpEventHandler = 'ch/dubernet/demopythonapi/simulation/events/JumpEventHandler'
SingEventHandler = 'ch/dubernet/demopythonapi/simulation/events/SingEventHandler'
SpeakEventHandler = 'ch/dubernet/demopythonapi/simulation/events/SpeakEventHandler'

In [ ]:
N_AGENTS = 100
N_TIME_STEPS = 100
N_TRIES = 100

In [ ]:
class Stopwatch:
    def __init__(self):
        self._records = []
    
    def start(self):
        self._start_time = time.time()
        
    def end(self, **attributes):
        t = time.time()
        self._records.append({**attributes, "time": t - self._start_time})
    
    def to_frame(self):
        return pd.DataFrame.from_records(self._records)

stopwatch = Stopwatch()

# Run using Java only

In [7]:
simulation = Simulation(N_AGENTS, N_TIME_STEPS)
simulation.getEvents().addEventHandler(BenchmarkEventHandler())

for i in range(N_TRIES):
    stopwatch.start()
    simulation.run()
    stopwatch.end(setting="pure java", n_agents=N_AGENTS, n_time_steps=N_TIME_STEPS)
    
print(stopwatch.to_frame().query("setting == \"pure java\"").time.mean())

pr = cProfile.Profile()
pr.enable()
simulation.run()
pr.disable()
pr.print_stats()

0.003193657398223877
         21 function calls in 0.001 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    0.001    0.001 <ipython-input-7-5ba06c0e9284>:13(<module>)
        1    0.000    0.000    0.000    0.000 <ipython-input-7-5ba06c0e9284>:14(<module>)
        2    0.000    0.000    0.000    0.000 codeop.py:132(__call__)
        2    0.000    0.000    0.000    0.000 hooks.py:142(__call__)
        2    0.000    0.000    0.000    0.000 hooks.py:207(pre_run_code_hook)
        2    0.000    0.000    0.000    0.000 interactiveshell.py:116(<lambda>)
        2    0.000    0.000    0.000    0.000 interactiveshell.py:1266(user_global_ns)
        2    0.000    0.000    0.001    0.001 interactiveshell.py:3259(run_code)
        2    0.000    0.000    0.000    0.000 ipstruct.py:125(__getattr__)
        2    0.000    0.000    0.000    0.000 {built-in method builtins.compile}
        2    0.000    0.000    

Pretty much the same as with JPype

# Naive Python Implementation

In [8]:
simulation = Simulation(N_AGENTS, N_TIME_STEPS)

class PythonBenchmarkHandler(jnius.PythonJavaClass):
    __javainterfaces__ = [JumpEventHandler, SingEventHandler, SpeakEventHandler]

    @jnius.java_method('()V')
    def notifyStart(self):
        pass
    
    @jnius.java_method('()V')
    def notifyEnd(self):
        pass
    
    @jnius.java_method('(Lch/dubernet/demopythonapi/simulation/events/JumpEvent;)V', name='handleEvent')
    def handleJumpEvent(self, event):
            event.getAgentId()
            event.getHeight_m()
            event.getTime()

            
    @jnius.java_method('(Lch/dubernet/demopythonapi/simulation/events/SingEvent;)V', name='handleEvent')
    def handleSingEvent(self, event):
            event.getAgentId()
            event.getSong()
            event.getTime()

            
    @jnius.java_method('(Lch/dubernet/demopythonapi/simulation/events/SpeakEvent;)V', name='handleEvent')
    def handleSpeakEvent(self, event):
            event.getAgentId()
            event.getMessage()
            event.getTime()

handler = PythonBenchmarkHandler()
simulation.getEvents().addEventHandler(handler)

for i in range(N_TRIES):
    stopwatch.start()
    simulation.run()
    stopwatch.end(setting="pure python", n_agents=N_AGENTS, n_time_steps=N_TIME_STEPS)
    
print(stopwatch.to_frame().query("setting == \"pure python\"").time.mean())
            
pr = cProfile.Profile()
pr.enable()
simulation.run()
pr.disable()
pr.print_stats()

1.2619202899932862
         210047 function calls in 1.376 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    20004    0.013    0.000    0.018    0.000 <frozen importlib._bootstrap>:416(parent)
    20004    0.008    0.000    0.028    0.000 <frozen importlib._bootstrap>:997(_handle_fromlist)
        1    0.000    0.000    0.000    0.000 <ipython-input-8-8b91bab5ebc6>:10(notifyEnd)
     3367    0.031    0.000    0.031    0.000 <ipython-input-8-8b91bab5ebc6>:14(handleJumpEvent)
     3269    0.062    0.000    0.062    0.000 <ipython-input-8-8b91bab5ebc6>:21(handleSingEvent)
     3364    0.035    0.000    0.035    0.000 <ipython-input-8-8b91bab5ebc6>:28(handleSpeakEvent)
        1    1.069    1.069    1.376    1.376 <ipython-input-8-8b91bab5ebc6>:46(<module>)
        1    0.000    0.000    0.000    0.000 <ipython-input-8-8b91bab5ebc6>:47(<module>)
        1    0.000    0.000    0.000    0.000 <ipython-input-8-8b91bab5ebc6>:6(no

That is roughly twice as bad as JPype... Let's see what the figure becomes once we reduce communication to the minimum.

# With Protocol Buffers

In [ ]:
simulation = Simulation(N_AGENTS, N_TIME_STEPS)

class PythonPbBenchmarkHandler:    
    def handleJumpEvent(self, event):
            event.agentId
            event.height_m
            event.time
    
    def handleSingEvent(self, event):
            event.agentId
            event.song
            event.time
            
    def handleSpeakEvent(self, event):
            event.agentId
            event.text
            event.time


simulation.getEvents().addEventHandler(api.demojnius.create_event_handler(PythonPbBenchmarkHandler()))

for i in range(N_TRIES):
    stopwatch.start()
    simulation.run()
    stopwatch.end(setting="pb python", n_agents=N_AGENTS, n_time_steps=N_TIME_STEPS)
    
print(stopwatch.to_frame().query("setting == \"pb python\"").time.mean())
            
pr = cProfile.Profile()
pr.enable()
simulation.run()
pr.disable()
pr.print_stats()